In [44]:
import pandas as pd
import nltk
from gensim.models import Word2Vec
import whois

## Alternative Domains finden mit Word2Vec (und eine Möglichkeit, ein kleines Taschengeld nebenbei zu verdienen)
Dieses Notebook ist Teil von <a href='https://datenverknoten.de/?p=296' target='_blank'>einem Artikel</a> auf www.datenverknoten.de.<br>
Der Korpus ist bereitgestellt von der Abteilung Automatische Sprachverarbeitung der Universität Leipzig. Er unterliegt einer CC BY-NC Lizenz, darf also <b>nicht kommerziell</b> verwendet werden. Hier geht es zur <a href='https://wortschatz.uni-leipzig.de/en/download/German' target='blank_'>Downloadseite</a>. Quelle: D. Goldhahn, T. Eckart & U. Quasthoff: Building Large Monolingual Dictionaries at the Leipzig Corpora Collection: From 100 to 200 Languages.
In: Proceedings of the 8th International Language Resources and Evaluation (LREC'12), 2012 

## GitHub
Der komplette Code samt trainiertem Modell (ohne Rohdaten, die aber mit Downloadlink) kann aus meinem <a href='https://github.com/datenverknoten/misc.git' target='blank_'>Repository im Verzeichnis misc_1 geklont werden</a>.

In [45]:
# Da ich den Korpus aus urheberrechtlichen Gründen nicht in meinem Repository bereitstellen möchte, 
# muss er selber runtergeladen werden. Es ist der Web-wrt-10M Korpus aus 2019.
# Wenn er im Data-Ordner vorhanden ist, wird er eingelesen.
data = pd.read_csv('Data/deu-de_web-wrt_2019_10M-sentences.txt',sep='\t',header=None).drop(columns=[2])

# Die Spalten erhalten einen Namen
data.columns = ['index','Sentence']

In [46]:
# So sieht der Korpus (die ersten drei Einträge aus)
data.head(3)

,index,Sentence
0,1,Aber allein diese drei Stichpunkte machen doch...
1,2,Aber auch dabei setzen wir die Hoffnung wieder...
2,3,Aber auch das – will ich hinzufügen – hat ja e...


In [ ]:
# Es werden ausschließlich Buchstaben und Leerzeichen behalten. Alle anderen Zeichen fliegen raus.
data['Sentence'] = data['Sentence'].str.replace('[^a-zA-ZäÄüÜöÖß ]', '', regex=True)
# Alle Sätze werden in lowercase umgewandelt.
data['Sentence'] = data['Sentence'].str.lower()

In [ ]:
# Die Sätze werden tokenized. So wird eine Liste von Worten erhalten. Sie werden als neue Spalte in
# den DataFrame gespeichert.
data['Tokenized_Sentence'] = data.apply(lambda row: nltk.word_tokenize(row['Sentence']), axis=1)

In [ ]:
# Hier wird die Spalte mit den tokenized Sätzen in eine Liste extrahiert. Es ist nun eine nested Liste.
sentences = list(data['Tokenized_Sentence'])

In [ ]:
# Das Word2Vec Model wird trainiert. Dieser Schritt kann, abhängig von der Leistung des verwendeten Rechners,
# mehrere Stunden dauern.
model = Word2Vec(sentences, min_count=1)

In [51]:
# Beispielhaft werden alle ähnlichen Worte zu 'Anwalt' ausgegeben. Die gefundenen 
# Worte stehen also in einem ähnlichen Zusammenhang in den Sätzen, wie eben das Wort 'Anwalt'.
# Da das Modell nur mit lowercase Worten trainiert wurde, muss das eingegebene Wort auch in lowercase sein.
model.wv.most_similar('anwalt')

[('rechtsanwalt', 0.8442790508270264),
 ('steuerberater', 0.7859551906585693),
 ('ombudsmann', 0.7800474762916565),
 ('fachanwalt', 0.7769657969474792),
 ('versicherungsmakler', 0.765956461429596),
 ('rechtsbeistand', 0.7305604815483093),
 ('strafverteidiger', 0.712742805480957),
 ('staatsanwalt', 0.7082164287567139),
 ('notar', 0.6978639960289001),
 ('makler', 0.6866521835327148)]

In [53]:
# Für Worte, die nicht bei der Modellerstellung genutzt wurden, kann auch kein ähnliches Wort
# gefunden werden. In diesem Fall wird ein KeyError ausgelöst. 
try:
    model.wv.most_similar('sdfsdf')
except KeyError as k:
    print("Keyerror")

Keyerror


In [ ]:
# Das Modell wird exportiert, da es im Webservice genutzt wird.
model.save('web_wrt.model')

In [10]:
# Hier soll beispielhaft der Ablauf gezeigt werden. Beim Besuch des Webservices muss ein Thema (fixed_word)
# und ein Name eingegeben werden, damit eine Kombination www.anwalt-schmidt.de entsteht.
fixed_word = 'Schmidt'
topic = 'Anwalt'

In [11]:
# Die ähnlichen Worte werden wieder aus dem Modell abgerufen.
results = model.wv.most_similar(topic.lower())

In [12]:
# Es entsteht eine Liste von Tupeln, aus denen nur das Wort extrahiert wird.
suggested_words = [x[0] for x in results]

In [40]:
# In dieser Schleife werden Domains erstellt. 
domains = []
for word in suggested_words:
    domains.append("www."+str(word)+"-"+str(fixed_word.lower())+".de")
    domains.append("www."+str(fixed_word.lower())+"-"+str(word)+".de")

In [41]:
# Mit whois wird abgerufen, ob die Seite registriert oder frei ist.
status_list = []
for dom in domains:
    try:
        domain = whois.whois(dom)
        status_list.append('Vergeben')
    except whois.parser.PywhoisError as pe:
        status_list.append('Frei')

In [42]:
# So sieht dann die Statusliste aus
status_list

['Vergeben',
 'Vergeben',
 'Vergeben',
 'Vergeben',
 'Frei',
 'Frei',
 'Vergeben',
 'Vergeben',
 'Vergeben',
 'Vergeben',
 'Frei',
 'Frei',
 'Vergeben',
 'Frei',
 'Frei',
 'Frei',
 'Vergeben',
 'Frei',
 'Vergeben',
 'Vergeben']

In [43]:
# Und so die Domains
domains

['www.rechtsanwalt-schmidt.de',
 'www.schmidt-rechtsanwalt.de',
 'www.steuerberater-schmidt.de',
 'www.schmidt-steuerberater.de',
 'www.ombudsmann-schmidt.de',
 'www.schmidt-ombudsmann.de',
 'www.fachanwalt-schmidt.de',
 'www.schmidt-fachanwalt.de',
 'www.versicherungsmakler-schmidt.de',
 'www.schmidt-versicherungsmakler.de',
 'www.rechtsbeistand-schmidt.de',
 'www.schmidt-rechtsbeistand.de',
 'www.strafverteidiger-schmidt.de',
 'www.schmidt-strafverteidiger.de',
 'www.staatsanwalt-schmidt.de',
 'www.schmidt-staatsanwalt.de',
 'www.notar-schmidt.de',
 'www.schmidt-notar.de',
 'www.makler-schmidt.de',
 'www.schmidt-makler.de']